# Download PISCES model output

 Here we download the hindcast ouput from the PISCES biogeochemical model corresponding to Tara Ocean samples. We use the [Copernicus Marine Toolbox](https://help.marine.copernicus.eu/en/collections/9080063-copernicus-marine-toolbox) python API with a tsv file containing the query coordinates as input (generated in a prior step) to download individual netcdf files for each coordinate query set.
 
 An alternative approach is to follow [this strategy](https://help.marine.copernicus.eu/en/articles/7970637-how-to-download-data-for-multiple-points-from-a-csv) where a Copernicus Marine data source is loaded as an xarray dataset using 'lazy-loading' mode. However, I found that opening the dataset remotely with lazy-loading was very slow. Subsetting the datset remotely, then downloading the subsetted product was much faster, albeit more clunky.

 **Note:** [you need to register with Copernicus Marine Data](https://data.marine.copernicus.eu/register) to use this resource.

## Global Ocean Biogeochemistry Hindcast

We will download all variables from the Global Ocean Biogeochemistry Hindcast for the grid point and time nearest to each Tara Oceans sample. After download these will be saved as a csv for later use.

- Product: [GLOBAL_MULTIYEAR_BGC_001_029](https://data.marine.copernicus.eu/product/GLOBAL_MULTIYEAR_BGC_001_029/description)  
- Dataset: `cmems_mod_glo_bgc_my_0.25deg_P1M-m`  
  
> The biogeochemical hindcast for global ocean is produced at Mercator-Ocean (Toulouse. France). It provides 3D biogeochemical fields since year 1993 at 1/4 degree and on 75 vertical levels. It uses PISCES biogeochemical model (available on the NEMO modelling platform). No data assimilation in this product.
>
> Latest NEMO version (v3.6_STABLE)
> - **Forcings:** FREEGLORYS2V4 ocean physics produced at Mercator-Ocean and ERA-Interim atmosphere produced at ECMWF at a daily frequency
> - **Outputs:** Daily (chlorophyll. nitrate. phosphate. silicate. dissolved oxygen. primary production) and monthly (chlorophyll. nitrate. phosphate. silicate. dissolved oxygen. primary production. iron. phytoplankton in carbon) 3D mean fields interpolated on a standard regular grid in NetCDF format. The simulation is performed once and for all.
> - **Initial conditions:** World Ocean Atlas 2013 for nitrate. phosphate. silicate and dissolved oxygen. > GLODAPv2 for DIC and Alkalinity. and climatological model outputs for Iron and DOC
> - **Quality/Accuracy/Calibration information:** See the [related QuID](https://documentation.marine.copernicus.eu/QUID/CMEMS-GLO-QUID-001-029.pdf)
> - **DOI (product):** https://doi.org/10.48670/moi-00019

# Import libraries & functions

We first import these libraries and define the `sort_dimension()` function to sort potential inverted axes. Here we are using Python `3.10.12`

In [1]:
import copernicusmarine as cm
import xarray as xr
import pandas as pd
import pathlib
import os

/home/shane/Documents/projects/method_dev/hogle_lab/omicsOceansEnvData/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Functions

In [2]:
def get_project_root():
    """
    Determines the root directory of the project by searching for specific markers.

    The function searches for the presence of a `.venv`, `renv`, or `.git` directory
    in the current working directory and its parent directories. The first directory
    containing any of these markers is considered the project root.

    Returns:
      pathlib.Path: The path to the project root directory if found, otherwise None.
    """
    current_path = pathlib.Path(os.getcwd())
    for path in [current_path, *current_path.parents]:
        if (path / ".venv").exists() or (path / "renv").exists() or (path / ".git").exists():
            return path
    return None


def get_filename_without_extension(file_path):
  """
  Extracts the filename without the extension from a given file path.

  Args:
    file_path: The path to the file.

  Returns:
    The filename without the extension.
  """
  filename = os.path.basename(file_path)
  name_without_extension, _ = os.path.splitext(filename)
  return name_without_extension


def read_netcdfs(files, dim):
    def process_one_path(path):
        """
        Processes a single file path to an xarray dataset.

        This function reads an xarray dataset from the given file path, assigns a new 
        value to the dataset which is the basename of the file (without extension), 
        and loads all data from the transformed dataset to ensure it can be used 
        after closing the original file.

        Parameters:
        path (str): The file path to the xarray dataset.

        Returns:
        xarray.Dataset: The processed xarray dataset with the new 'query_id' attribute.
        """
        # use a context manager, to ensure the file gets closed after use
        with xr.open_dataset(path) as ds:
            # assign new value which is basename of the file that was read
            ds = ds.assign(query_id=get_filename_without_extension(path))
            # load all data from the transformed dataset, to ensure we can
            # use it after closing each original file
            ds.load()
            return ds

    paths = sorted(files)
    datasets = [process_one_path(p) for p in paths]
    combined = xr.concat(datasets, dim, coords='minimal')
    return combined

# Read input csv file of Tara coordinates

Also perform some small formatting to generate unique identifiers for each `tara_station` + `depth` combination

In [3]:
# get the project root
proj_root=get_project_root()

# Read the CSV in a pandas dataframe
query_coords = pd.read_csv(
    str(proj_root)+"/data/tara/pisces_subsets/pisces_tara_grid_map.csv")

# subset to columns of interest
query_coords = query_coords.loc[:, ['tara_station',
                                    'latitude', 'longitude', 'date_time', 'depth']] 

# make a new unique identifer
query_coords["query_id"] = query_coords["tara_station"] + "_d" + query_coords["depth"].astype('int').astype('str')

# Convert columns into right format
query_coords["date_time"] = pd.to_datetime(
    query_coords["date_time"])

# drop rows with duplicate information
query_coords = query_coords.drop_duplicates()

query_coords

,tara_station,latitude,longitude,date_time,depth,query_id
0,TARA_003,36.672,-10.421,2009-09-13,9.0,TARA_003_d9
1,TARA_004,36.563,-6.553,2009-09-15,9.0,TARA_004_d9
2,TARA_004,36.563,-6.553,2009-09-15,40.0,TARA_004_d40
3,TARA_005,36.030,-4.405,2009-09-20,9.0,TARA_005_d9
4,TARA_005,36.030,-4.405,2009-09-20,68.0,TARA_005_d68
...,...,...,...,...,...,...
301,TARA_208,69.107,-51.578,2013-10-20,5.0,TARA_208_d5
302,TARA_209,64.728,-53.467,2013-10-23,5.0,TARA_209_d5
303,TARA_209,64.728,-53.467,2013-10-23,200.0,TARA_209_d200
304,TARA_210,61.544,-55.985,2013-10-27,5.0,TARA_210_d5


# Download from Copernicus marine

In [18]:
%%time
for rec in query_coords.to_dict('records'):
  nc_path = pathlib.Path(str(proj_root)+ str('/data/tara/pisces_subsets/') + rec['query_id'] + '.nc')
  if not nc_path.exists():
    cm.subset(
      dataset_id="cmems_mod_glo_bgc_my_0.25deg_P1M-m",
      start_datetime=str(rec['date_time']),
      end_datetime=str(rec['date_time']),
      minimum_longitude=rec['longitude'],
      maximum_longitude=rec['longitude'],
      minimum_latitude=rec['latitude'],
      maximum_latitude=rec['latitude'],
      minimum_depth=rec['depth'],
      maximum_depth=rec['depth'],
      coordinates_selection_method="nearest",
      output_filename=rec['query_id'],
      output_directory=str(proj_root) + str('/data/tara/pisces_subsets'),
      disable_progress_bar=False
  )

CPU times: user 36min 50s, sys: 1min 49s, total: 38min 40s
Wall time: 1h 42min 57s


# Read and format netcdf output

You would think that [`xarray.open_mfdataset`](https://docs.xarray.dev/en/stable/generated/xarray.open_mfdataset.html#xarray.open_mfdataset) is the best way to read all the netcdf files from the last step. However, this is very slow for opening lots of small netcdf files. 

The more efficient approach is to just use a custom function `read_netcdfs` defined above.

In [10]:
# get listing of all files in the pisces_subsets
flist=[x for x in pathlib.Path(
    str(proj_root) + str('/data/tara/pisces_subsets/')).glob('*.nc') if x.is_file()]

In [11]:
# read and concatenate nc files 
datasets = read_netcdfs(flist, "latitude")

Convert the xarray concatenated datasets to a dataframe ([`to_dataframe`](https://docs.xarray.dev/en/latest/generated/xarray.Dataset.to_dataframe.html#xarray.Dataset.to_dataframe))

In [12]:
# convert concatenated xarrays to pandas dataframe
dataframes = datasets.to_dataframe(
    dim_order=['latitude', 'longitude', 'depth', 'time']).dropna()

Save as tsv for later use

In [14]:
# write the output 
dataframes.to_csv(
    str(proj_root) + str('/data/tara/pisces_subsets/pisces_tara_subset_filt.tsv'), sep='\t')

## Work environment

In [ ]:
import session_info
session_info.show()